In [3]:
# connect to test.db sqlite, create session, then run "get all users" with session.Query
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from models import Base, Cast, User, Reaction, Location, parent_association
import pandas as pd


engine = create_engine('sqlite:///test.db')
Session = sessionmaker(bind=engine)
session = Session()


In [7]:
"""
Here are all the available models:
parent_association = Table('parent_association', Base.metadata,
                           Column('parent_hash', String,
                                  ForeignKey('casts.hash')),
                           Column('cast_hash', String,
                                  ForeignKey('casts.hash'))
                           )


# figure out ancestor hashes later
class Cast(Base):
    __tablename__ = 'casts'
    hash = Column(String, primary_key=True)
    thread_hash = Column(String, ForeignKey(
        'casts.hash'))
    parent_hash = Column(String, ForeignKey(
        'casts.hash'), nullable=True)
    text = Column(String)
    timestamp = Column(BigInteger)
    author_fid = Column(BigInteger, ForeignKey(
        'users.fid'))
    author = relationship('User', back_populates='casts')
    reactions = relationship('Reaction', back_populates='target')
    children_hashes = relationship("Cast", secondary=parent_association,
                                   primaryjoin=(
                                       hash == parent_association.c.parent_hash),
                                   secondaryjoin=(
                                       hash == parent_association.c.cast_hash),
                                   backref="parent_casts")


class Reaction(Base):
    __tablename__ = 'reactions'
    hash = Column(String, primary_key=True)
    reaction_type = Column(String)  # like & recast
    timestamp = Column(BigInteger)
    target_hash = Column(String, ForeignKey('casts.hash'))
    author_fid = Column(BigInteger, ForeignKey('users.fid'))
    target = relationship('Cast', back_populates='reactions')


class Location(Base):
    __tablename__ = 'locations'
    place_id = Column(String, primary_key=True)
    description = Column(String)
    users = relationship('User', backref='location')


class User(Base):
    __tablename__ = 'users'
    fid = Column(BigInteger, primary_key=True)
    username = Column(String)
    display_name = Column(String)
    verified = Column(Boolean, default=False)
    pfp_url = Column(String, nullable=True)
    follower_count = Column(BigInteger)
    following_count = Column(BigInteger)
    bio_text = Column(String, nullable=True)
    location_place_id = Column(String, ForeignKey(
        'locations.place_id'), nullable=True)
    casts = relationship('Cast', back_populates='author')

"""

'\nHere are all the available models:\n'

In [14]:
# get all users where they have location with description that contains "USA"
# description_location is from the Location model
# location_place_id is from the User model

locations = session.query(Location).filter(
    Location.description.contains("USA")).all()

# print users with locations
users = session.query(User).filter(
    User.location_place_id.in_([location.place_id for location in locations])).all()

df = pd.DataFrame([user.__dict__ for user in users])
# print(df)

# make df with username, location column

new_df = pd.DataFrame(columns=["username", "location"])
for user in users:
    new_df = pd.concat([new_df, pd.DataFrame({"username": [user.username], "location": [user.location.description]})], ignore_index=True)

print(new_df)

            username                location
0           jplevyak  San Francisco, CA, USA
1           jerblack       New York, NY, USA
2     mm-fc-00000033  San Francisco, CA, USA
3             hirsch        Houston, TX, USA
4              scorz         Denver, CO, USA
5         chadfowler         Denver, CO, USA
6            geoveza         Denver, NY, USA
7              naman  San Francisco, CA, USA
8                jey         Austin, TX, USA
9                 by    Los Angeles, CA, USA
10           snuglas      San Diego, CA, USA
11             bayka        Seattle, WA, USA
12             vladn          Miami, FL, USA
13     thelafffinman      San Diego, CA, USA
14         ericnagel        Buffalo, NY, USA
15          das-mchl       New York, NY, USA
16          newtroll   Montana City, MT, USA
17      constantinos        Bozeman, MT, USA
18           hillale       Rockford, IL, USA
19       kelvintiger         Denver, CO, USA
20              taek         Denver, CO, USA
21        

In [20]:
casts_with_no_parent = session.query(
    Cast).filter(Cast.parent_hash == None).all()

all_associations = session.query(parent_association).all()

all_casts = session.query(Cast).all()

print(len(all_casts) - len(casts_with_no_parent) == len(all_associations))
print(len(all_casts), len(casts_with_no_parent), len(all_associations))


True
1000 304 696
